<a href="https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/once_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding=utf-8
#@title 	&#8592;Click [ ▶ ] to Start. { display-mode: "form" }
#@markdown Clean Your Wiseman Score
# Class = "5a" #@param ["4a", "4b", "4c", "4d", "5a", "5b", "5c", "5d", "6a", "6b", "6c", "6d"]
# Class_number = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31" , "32", "33", "34", "35", "36", "37", "38", "39", "40"]
username = "5a29" #@param {type:"string"}
#@markdown 輸入需要清除的課程數量
number = 10 #@param {type:"slider", min:0, max:100, step:1}
from google.colab import widgets
from google.colab import output
try:
  from selenium import webdriver
except:
  # 清除輸出Log
  !apt -q -qq update
  !apt -q -qq install chromium-chromedriver
  !pip -q install selenium
  from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
options.add_argument('log-level=3')
# log-level:
# Sets the minimum log level.
# Valid values are from 0 to 3:
#	 INFO = 0,
#	 WARNING = 1,
#	 LOG_ERROR = 2,
#	 LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
output.clear()

driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=tiaccwhf.%s&password=2020'%username)
# driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
# 等待頁面加載完畢
try:
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card"]/div[@class="card-body"]/div[@class="text-right"]/h1[@class="font-light mb-0 d-inline-block d-md-block"]')))
except:
  pass
student_name=str(driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('textContent'))#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
before_score=int(str(driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]/h1').get_attribute('textContent')).replace('&nbsp;',''))#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
if before_score == 0:
  print('%s Score:%d'%(student_name,before_score))
  pass
# 進入課程列表
driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
# link element
wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="card-body"]/div[@class="table-responsive"]/table[@class="table table-striped"]')))
# 獲取所有鏈接和標題
uncomplete=[]
# 獲取tr橫向表格框
for tr in driver.find_elements_by_xpath('//div[@class="card-body"]/div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr'):
  try:
    # 在tr框内尋找鏈接
    a = tr.find_element_by_xpath('.//a[contains(@href,"selectLesson.do?id=")]')
    # if 狀態非New
    if tr.find_element_by_xpath('.//td[@class="small align-middle"]//*[@title]').get_attribute('title')!='New':
      origin_status = tr.find_element_by_xpath('.//td[@class="small align-middle"]//*[@title]').get_attribute('title')
      # 獲取標題
      title = a.get_attribute('textContent')
      # 獲取鏈接
      link = a.get_attribute('href')
      # 加入列表
      uncomplete.append({'title':title,'link':link})
  except:
    pass
now_lesson=0
total_lessons=len(uncomplete)
print('%sScore:%d'%(student_name,before_score))
grid = widgets.Grid((total_lessons+1), 3, header_row=True)
with grid.output_to(0, 1):print('Status')
with grid.output_to(0, 2):print('Title')
# 在列表範圍内循環
numb=0
for i in reversed(uncomplete):
  if numb>=number:
    break
  now_lesson+=1
  with grid.output_to(now_lesson, 0):print('%d/%d'%(now_lesson,total_lessons))
  with grid.output_to(now_lesson, 1):print(origin_status)
  with grid.output_to(now_lesson, 2):print('%s(%s)'%(i['title'],i['link']))
  # 進入鏈接
  driver.get(i['link'])
  wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
  # Loading time
  driver.switch_to.frame('course')
  # 等待頁面加載完畢
  # wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="head course-head"]/div[@class="course-head-text"]')))
  wait.until(EC.presence_of_element_located((By.TAG_NAME,'label')))
  # Return top
  driver.switch_to.default_content()
  driver.execute_script('''
  doCourseAPIKeepAlive=function(){return false;};
  API.cmiData["cmi.core.lesson_status"]="new";
  var formData = new FormData();
  formData.set('token', API.token);
  formData.set('data', API.cmiData.toJSONString());
  navigator.sendBeacon("commit.do", formData);
  // API.cmiData["cmi.core.score.raw"]=100;
  // API.cmiData["cmi.core.lesson_status"]="completed";
  // navigator.sendBeacon("commit.do", formData);
  navigator.sendBeacon=function(){return false;};
  ''')
  with grid.output_to(now_lesson, 1):
    grid.clear_cell()
    print('New')
  numb+=1
# 進入分數頁面
driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/dashboard.shtml')
# 等待頁面加載完畢
wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card-body"]/div[@class="text-right"]/h1')))
# 完成后的分數
after_score=int(str(driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]/h1').get_attribute('textContent')).replace('&nbsp;',''))
# 登出Wiseman
driver.get('https://lms.wiseman.com.hk/lms/user/logout.do')
print('%s[%d=>%d]'%(student_name,before_score,after_score))
driver.quit()

5A-29-YAO CHUN SHUM Score:4475


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

New


<IPython.core.display.Javascript object>

5A-29-YAO CHUN SHUM [4475=>3950]
